In [1]:
import numpy as np
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.layers import *
from tensorflow.keras.models import *

2023-04-03 11:02:41.894210: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-03 11:02:42.020505: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-03 11:02:42.047509: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a suppor

# ResNet

In [5]:
def conv_bn_relu(x, filters, kernel_size, strides=1, padding='same'):
    x = Conv2D(filters, kernel_size, strides=strides, padding=padding)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x

def identity_block(x, filters):
    shortcut = x
    x = conv_bn_relu(x, filters, 1)
    x = conv_bn_relu(x, filters, 3)
    x = conv_bn_relu(x, filters * 4, 1)
    x = Add()([x, shortcut])
    x = ReLU()(x)
    return x

def projection_block(x, filters, strides):
    shortcut = conv_bn_relu(x, filters * 4, 1, strides)
    x = conv_bn_relu(x, filters, 1, strides)
    x = conv_bn_relu(x, filters, 3)
    x = conv_bn_relu(x, filters * 4, 1)
    x = Add()([x, shortcut])
    x = ReLU()(x)
    return x

def resnet(input_shape, num_classes, num_layers):
    if num_layers == 50:
        num_blocks = [3, 4, 6, 3]   
    elif num_layers == 101:
        num_blocks = [3, 4, 23, 3]   
    elif num_layers == 152:
        num_blocks = [3, 8, 36, 3]

    conv2_x, conv3_x, conv4_x, conv5_x = num_blocks

    inputs = Input(shape=input_shape)
    x = ZeroPadding2D(padding=(3, 3))(inputs)
    x = conv_bn_relu(x, 64, 7, strides=2)
    x = MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')(x)

    x = projection_block(x, 64, strides=1)
    for _ in range(conv2_x - 1):
        x = identity_block(x, 64)

    x = projection_block(x, 128, strides=2)
    for _ in range(conv3_x - 1):
        x = identity_block(x, 128)

    x = projection_block(x, 256, strides=2)
    for _ in range(conv4_x - 1):
        x = identity_block(x, 256)

    x = projection_block(x, 512, strides=2)
    for _ in range(conv5_x - 1):
        x = identity_block(x, 512)

    x = GlobalAveragePooling2D()(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)
    return model

model = resnet(input_shape=(224,224,3),  num_classes=200, num_layers=152)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_4[0][0]']                
                                                                                                  
 conv2d_78 (Conv2D)             (None, 115, 115, 64  9472        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                                            

## Ensemble 기법 도입

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Add
from tensorflow.keras.models import Model

# Define Residual Network architecture
def residual_network(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv2D(32, (3, 3), padding='same', activation='relu')(inputs)
    x = Conv2D(32, (3, 3), padding='same', activation='relu')(x)
    residual = x
    x = Conv2D(32, (3, 3), padding='same', activation='relu')(x)
    x = Add()([x, residual])
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    outputs = Dense(10, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Train multiple Residual Networks
num_networks = 3
input_shape = (32, 32, 3)
num_classes = 10
epochs = 10
batch_size = 32

models = []
for i in range(num_networks):
    model = residual_network(input_shape)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val))
    models.append(model)

# Make predictions
predictions = []
for model in models:
    predictions.append(model.predict(X_val))

# Combine predictions
ensemble_prediction = tf.reduce_mean(predictions, axis=0)

# Evaluate ensemble
ensemble_score = model.evaluate(X_test, y_test)

# Test ensemble
test_predictions = []
for model in models:
    test_predictions.append(model.predict(X_test))

ensemble_test_prediction = tf.reduce_mean(test_predictions, axis=0)
ensemble_test_score = model.evaluate(X_test, y_test)